# FAISS DB

Facebook AI Similarity Search

In [1]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------- ----- 15.7/18.2 MB 82.7 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 63.7 MB/s  0:00:00


### 영화 추천 시스템

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\Playdata\AppData\Local\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
movies = [
    {"id": 0, "title": "인셉션", "description": "꿈속에서 다른 사람의 생각을 조작하는 스토리"},
    {"id": 1, "title": "어벤져스: 엔드게임", "description": "어벤져스가 타노스를 물리치기 위한 마지막 전투를 벌이는 이야기"},
    {"id": 2, "title": "기생충", "description": "부유한 가정과 가난한 가정의 충격적인 만남을 그린 사회적 드라마"},
    {"id": 3, "title": "해리 포터와 마법사의 돌", "description": "마법의 세계에 입학한 소년 해리 포터의 첫 번째 모험"},
    {"id": 4, "title": "반지의 제왕: 왕의 귀환", "description": "반지를 파괴하기 위한 최후의 전투가 벌어지는 이야기"},
    {"id": 5, "title": "스타워즈: 제국의 역습", "description": "제국의 추격을 피해 도망치는 반란군들의 이야기"},
    {"id": 6, "title": "타이타닉", "description": "침몰하는 타이타닉 호에서의 사랑과 비극적인 운명을 그린 이야기"},
    {"id": 7, "title": "쇼생크 탈출", "description": "불법적인 이유로 감옥에 갇힌 남자가 탈출을 계획하는 이야기"},
    {"id": 8, "title": "인터스텔라", "description": "우주를 넘어서 인류의 생존을 위한 탐험을 그린 이야기"},
    {"id": 9, "title": "블랙 팬서", "description": "와칸다 왕국의 새로운 왕이 되어가는 이야기를 그린 슈퍼히어로 영화"},
    {"id": 10, "title": "어드벤처 타임", "description": "모험과 상상력으로 가득한 세계에서 주인공들이 펼치는 이야기"},
    {"id": 11, "title": "혹성 탈출", "description": "인간과 유인원의 관계를 그린 고전적인 SF 영화"},
    {"id": 12, "title": "죽은 시인의 사회", "description": "고등학교 교사와 학생들이 펼치는 삶의 의미를 찾는 이야기"},
    {"id": 13, "title": "올드보이", "description": "13년간 감금된 남자가 복수를 위해 싸우는 미스터리 영화"},
    {"id": 14, "title": "아이언맨", "description": "억만장자 천재 발명가가 슈퍼히어로로 변신하는 이야기"},
    {"id": 15, "title": "포레스트 검프", "description": "특별한 능력 없이 살아온 한 남자의 삶을 그린 감동적인 이야기"},
    {"id": 16, "title": "카카오", "description": "기업 카카오의 창업과 성공 스토리를 그린 영화"},
    {"id": 17, "title": "디지털 포레스트", "description": "인터넷과 디지털 기술로 뒤얽힌 사회와 인간의 관계를 그린 영화"},
    {"id": 18, "title": "미션 임파서블: 고스트 프로토콜", "description": "특수 요원들이 전 세계에서 벌어지는 테러를 막기 위해 고군분투하는 이야기"},
    {"id": 19, "title": "어둠의 경로", "description": "밤마다 악몽을 겪는 주인공이 자신만의 진실을 찾아가는 스릴러 영화"}
]


In [4]:
import numpy as np

descriptions = [movie['description'] for movie in movies]
desc_embed = np.array([model.encode(description) for description in descriptions], dtype='float32')

In [6]:
import faiss

dim = desc_embed.shape[1]
idx = faiss.IndexFlatL2(dim)
idx.add(desc_embed)

In [7]:
query_text = '탈출하는 이야기'
query_embed = np.array([model.encode(query_text)], dtype='float32')

In [8]:
top_n = 5
distance, indices = idx.search(query_embed, top_n)

print(distance)
print(indices)

[[0.8666453 0.8801925 0.9008679 0.9916291 1.0403186]]
[[ 0 13  4  3 18]]


In [10]:
for i in range(top_n):
    movie_id = indices[0][i]
    print(f'{i+1}번째 추천: {movies[movie_id]["title"]} (유사도 거리: {distance[0][i]:.2f})')

1번째 추천: 인셉션 (유사도 거리: 0.87)
2번째 추천: 올드보이 (유사도 거리: 0.88)
3번째 추천: 반지의 제왕: 왕의 귀환 (유사도 거리: 0.90)
4번째 추천: 해리 포터와 마법사의 돌 (유사도 거리: 0.99)
5번째 추천: 미션 임파서블: 고스트 프로토콜 (유사도 거리: 1.04)


### 사용자 맞춤 뉴스 추천

In [11]:
!pip install bs4

In [12]:
# 크롤링
import requests
from bs4 import BeautifulSoup

keyword = '2025.09.03'
url = f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query={keyword}"

response = requests.get(url)
html = response.text

bs = BeautifulSoup(html, 'html.parser')

news_contents = bs.select('.fds-news-item-list-tab > div')

news_list = []

for idx, news_content in enumerate(news_contents):
    title_tag = news_content.select_one('span.sds-comps-text-type-headline1')
    title = title_tag.text    
    news_list.append(title)

In [15]:
news_list

['30년간 출생아 47만명 감소…부모 평균 출산연령 5세↑',
 '더 기아 EV5 실내',
 '정부, 수출기업 13.6조 긴급 수혈…대미 수출 총력전',
 "한수원, 세르비아 에너지시장 공략…원자력·수소 협력 '시동'",
 "문체부, '2025년 젊은 건축가상'에 김선형 교수 등 3개팀 선정",
 '[US오픈] 조코비치, 세계 2위 알카라스와 준결승서 격돌',
 '동해해경청, 9일까지 강릉시에 긴급 급수 1668t 지원',
 '의정부시, 2025년 적극행정 토론형 심화교육 개최',
 '한국 귀신 흥행사',
 '[기획] 꽃·예술, 막걸리·야경…고양시 9월 내내 축제 한마당']

In [13]:
# 임베딩 모델 로드 및 임베딩 처리
model = SentenceTransformer("all-MiniLM-L6-v2")
news_embed = np.array([model.encode(news_title) for news_title in news_list], dtype="float32")

c:\Users\Playdata\AppData\Local\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
# FAISS index 생성 및 데이터 추가
dim = news_embed.shape[1]
idx = faiss.IndexFlatL2(dim)
idx.add(news_embed)

In [16]:
# 질의 텍스트 임베딩
query_text = '축제'
query_embedding = np.array([model.encode(query_text)], dtype='float32')

In [17]:
# 검색
distances, indices = idx.search(query_embedding, 2)

In [18]:
for i in range(2):
    news_id = indices[0][i]
    print(f'{news_list[news_id]} (유사도 거리: {distances[0][i]:.2f})')

한국 귀신 흥행사 (유사도 거리: 0.87)
[기획] 꽃·예술, 막걸리·야경…고양시 9월 내내 축제 한마당 (유사도 거리: 0.90)
